In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
# di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import hail as hl
hl.init(tmp_dir='/net/scratch/people/plggosborcz')

In [ ]:
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()

import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain
import statistics as stat

import bokeh.palettes
import itertools

In [ ]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

## note - the paths in this notebook are outdated

## count genes that were included in the SKAT test:

In [ ]:
genes = ['DCC', 'RBFOX', 'SLC30A9', 'DCAF4L1', 'SORCS3', 'KCNQ5', 'KCNQ-IT1', 'APOPT1', 'C14orf2', 'NAA11', 'NEGR1',
        'CHADL', 'SOX5', 'PANK2', 'COL27A1', 'PDGFB', 'CELSR3', 'OPA1', 'FBN2', 'WWC1', 'NIPBL', 
             'FN1', 'FBN2', 'SLITRK1', 'SLITRK2', 'SLITRK3', 'SLITRK4', 'SLITRK5', 'SLITRK6', 
             'HDC', 'OPRK1', 'PCDH10', 'NTSR2', 'OPRK1', 'CHD8', 'SCUBE1', 'PNKD', 'CNTNAP2', 'MOG', 
             'DRD2', 'DRD3', 'DRD4', 'DRD5', 'DAT1', 'DBH', 'HTR2A', 'TPH2', 'EAAT1', 'SAPAP3',
            'CTNNA3', 'NLGN4', 'FSCB', 'IMMP2L', 'NRXN1', 'AADAC', 'DBH', 'MAOA', 'HTR1A', 'HTR2C', 'SLC6A4',
             'TPH2', 'COL27A1', '5-HTTLPR', 'EAAT1', 'COL8A1', 'KCNE1', 'KCNE2',
         'RICTOR', 'WWC1', 'CELSR3, NIPBL', 'FN1', 'PNKD', 'CDH26', 'CADM2', 'OPCML', 'CDH9',
         'NCAM2', 'CD47', 'CDH5', 'CADM4', 'C1QBP', 'CTTN', 'LSAMP',
         'PKP4', 'PCDH1', 'CNTNAP2', 'MBP', 'GABBR2', 'GABBR2', 'GRIK4', 'NCR1', 'FLT3', 'IL12A', 'HDAC9',
         'CD180', 'CDH26', 'NCAM2', 'NTM', 'ROBO2']


mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-gnomad-sex.mt')


mt = mt.filter_rows(hl.any(lambda x: hl.literal(genes).contains(x), mt.nearest_genes_20kb))
mt.count()

In [ ]:
mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/skat-gene-variants.mt')

In [ ]:
mt = mt.filter_rows(mt.cadd > 10)
mt.count()

In [ ]:
mt = mt.filter_rows(mt.cadd > 20)
mt.count()

In [ ]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/skat-gene-variants.mt')

In [ ]:
mt.aggregate_rows(hl.agg.counter(mt.gnomad_v3.most_severe_consequence))

## export table with all of investigated variants - summary stats

In [ ]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/skat-gene-variants.mt')

In [ ]:
#reasigned phenotypes:
pheno = hl.import_table('/net/archive/groups/plggneuromol/GTS-analysis/analysis/GTS-coded-corrected-june-2021.csv', delimiter=',', impute=True,  quote='\"')

In [ ]:
pheno = pheno.key_by(pheno.ID)

In [ ]:
mt = mt.annotate_cols(phenotypes = pheno[mt.s])

In [ ]:
mt = mt.annotate_rows(cases_with_gts = hl.agg.group_by(mt.phenotypes.family, hl.agg.count_where(mt.phenotypes.disease == 'YES')),
                      variants_with_gts = hl.agg.group_by((mt.phenotypes.family, mt.phenotypes.disease), hl.agg.count_where(mt.GT.is_non_ref())),
                      carriers_in_families = hl.agg.group_by(mt.s, hl.agg.count_where(mt.GT.is_non_ref())),
                      variants_in_sporadic_cases = hl.agg.group_by((mt.phenotypes.family, mt.phenotypes.disease), hl.agg.counter(mt.GT.is_non_ref()))
                     )

In [ ]:
mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-with-skat.mt')

In [ ]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-with-skat.mt')

In [ ]:
mt = mt.annotate_rows(sporadic_cases_non_refs = hl.if_else(mt.variants_in_sporadic_cases[('.','YES')].contains(True), 
                                                                mt.variants_in_sporadic_cases[('.','YES')][True],
                                                                0),
                             sporadic_cases_refs = hl.if_else(mt.variants_in_sporadic_cases[('.','YES')].contains(False), 
                                                                mt.variants_in_sporadic_cases[('.','YES')][False],
                                                                0))

In [ ]:
mt.checkpoint('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-with-skat-2.mt')

In [ ]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-with-skat-2.mt')

In [ ]:
a = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-for-skat-may.mt')
b = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-test-may.mt')

samples = a.s.collect() + b.s.collect()

In [ ]:
len(samples)

In [ ]:
mt = mt.filter_cols(hl.literal(samples).contains(mt.s))

In [ ]:
mt.count()

In [ ]:
mt = mt.annotate_rows(variants_in_gnomad = hl.agg.group_by((mt.s.contains('gnomad')), hl.agg.counter(mt.GT.is_non_ref())))

In [ ]:
mt = mt.filter_cols(mt.s.contains('gnomad'), keep = False)

In [ ]:
mt = mt.annotate_rows(variants_in_familial = hl.agg.group_by((mt.phenotypes.disease), hl.agg.counter(mt.GT.is_non_ref())))

In [ ]:
mt = mt.annotate_rows(controls_non_refs = hl.if_else(mt.variants_in_familial['NO'].contains(True), 
                                                                mt.variants_in_familial['NO'][True],
                                                                0),
                              controls_refs = hl.if_else(mt.variants_in_familial['NO'].contains(False), 
                                                                mt.variants_in_familial['NO'][False],
                                                                0),
                              cases_non_refs = hl.if_else(mt.variants_in_familial['YES'].contains(True), 
                                                                mt.variants_in_familial['YES'][True],
                                                                0),
                              cases_refs = hl.if_else(mt.variants_in_familial['YES'].contains(False), 
                                                                mt.variants_in_familial['YES'][False],
                                                                0))

In [ ]:
mt.checkpoint('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-with-skat-3.mt')

In [ ]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-with-skat-3.mt')

In [ ]:
mt = mt.filter_rows(mt.cadd > 0)

In [ ]:
mt = mt.rows()

In [ ]:
mt = mt.annotate(gnomads_non_ref = hl.if_else(mt.variants_in_gnomad[True].contains(True), 
                                                               mt.variants_in_gnomad[True][True],
                                                                0))

In [ ]:
mt = mt.select(mt.gnomad_v3.most_severe_consequence,
                    mt.hpo, mt.cadd, mt.nearest_genes_20kb, mt.sporadic_cases_non_refs, mt.sporadic_cases_refs, mt.controls_non_refs,
               mt.controls_refs, mt.cases_non_refs, mt.cases_refs, mt.gnomads_non_ref)

mt = mt.to_pandas()

mt.to_csv('/net/archive/groups/plggneuromol/GTS-analysis/data/supplementary-table-T1.csv')

In [ ]:
mt[1:1000].to_csv('/net/archive/groups/plggneuromol/GTS-analysis/data/subset-supplementary-table-T1.csv')

## investigate the asignment in families

In [ ]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-gnomad-sex.mt')

top_genes = ['HDC', 'CHADL', 'MAOA', 'NAA11', 'PCDH10']

mt = mt.filter_rows(hl.any(lambda x: hl.literal(top_genes).contains(x), mt.nearest_genes_20kb))
mt = mt.filter_rows(mt.cadd > 10)

In [ ]:
#reasigned phenotypes:
pheno = hl.import_table('/net/archive/groups/plggneuromol/GTS-analysis/analysis/GTS-coded-corrected-june-2021.csv', delimiter=',', impute=True,  quote='\"')

In [ ]:
pheno = pheno.key_by(pheno.ID)

In [ ]:
mt = mt.annotate_cols(phenotypes = pheno[mt.s])

In [ ]:
mt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/top-variants.mt')

In [ ]:
top = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/top-variants.mt')

In [ ]:
mt_test = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/mt_test_10.mt')

In [ ]:
false_pos = np.load('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/results/numpy/may-false_pos5cadd10.npy')
true_pos = np.load('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/results/numpy/may-true_pos5cadd10.npy')

In [ ]:
top_test = top.filter_cols(hl.set(['.', 'gnomad']).contains(top.phenotypes.family), keep = False)

In [ ]:
samples = mt_test.s.collect()

In [ ]:
top_test = top_test.filter_cols(hl.set(samples).contains(top_test.s))

In [ ]:
top_test.count()

In [ ]:
families = mt_test.phenotypes.family.collect()

In [ ]:
np.where((false_pos < 0.45) & (true_pos > 0.65))

In [ ]:
false_pos[11204]

In [ ]:
true_pos[11204]

In [ ]:
false_pos[10759]

In [ ]:
true_pos[10759]

In [ ]:
test_asignment = np.load('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/results/numpy/may-test_asignment5cadd10.npy')

In [ ]:
len(test_asignment)

In [ ]:
test_asignment

In [ ]:
np.linspace(-25,20,20000)[11204]

In [ ]:
best_asignment = test_asignment > np.linspace(-25,20,20000)[11204]

In [ ]:
mt_test = mt_test.annotate_cols(phenotypes = pheno[mt_test.s])

In [ ]:
nonCTS = mt_test.phenotypes.nonCTD.collect()

In [ ]:
ASD = mt_test.phenotypes.add_pheno.contains('ASD').collect()

In [ ]:
group = mt_test.phenotypes.phenotype.collect()

In [ ]:
np.sum((np.array(group) == 'GTS'))

In [ ]:
np.sum((np.array(group) == 'tics') & best_asignment)

In [ ]:
14/42

In [ ]:
np.sum((np.array(nonCTS) == 'YES') & best_asignment)

In [ ]:
4/17

In [ ]:
33/47

In [ ]:
np.sum((np.array(group) == 'GTS') & best_asignment)

In [ ]:
18/47

In [ ]:
np.sum((np.array(group) == '.') & best_asignment)

In [ ]:
(56-15)/56

In [ ]:
top_test.aggregate_cols(hl.agg.filter((top_test.phenotypes.family == 'W'), hl.agg.counter(top_test.phenotypes.phenotype)))

In [ ]:
fam_s = top_test.s.collect()

In [ ]:
np.array(fam_s)[np.array(families) == 'W']

In [ ]:
np.array(best_asignment)[np.array(families) == 'W']

In [ ]:
np.array(group)[np.array(families) == 'W']

In [ ]:
hdc_coding = hl.agg.filter((hl.str(top_test.nearest_genes_20kb).contains('HDC') & ((top_test.gnomad_v3.most_severe_consequence == 'missense_variant') |
                                                                            (top_test.gnomad_v3.most_severe_consequence == 'synonymous_variant'))),
                           hl.agg.count_where(top_test.GT.is_non_ref()))

chadl_coding = hl.agg.filter((hl.str(top_test.nearest_genes_20kb).contains('CHADL') & ((top_test.gnomad_v3.most_severe_consequence == 'missense_variant') |
                                                                            (top_test.gnomad_v3.most_severe_consequence == 'synonymous_variant'))),
                           hl.agg.count_where(top_test.GT.is_non_ref()))

maoa_coding = hl.agg.filter((hl.str(top_test.nearest_genes_20kb).contains('MAOA') & ((top_test.gnomad_v3.most_severe_consequence == 'missense_variant') |
                                                                            (top_test.gnomad_v3.most_severe_consequence == 'synonymous_variant'))),
                           hl.agg.count_where(top_test.GT.is_non_ref()))
naa11_coding = hl.agg.filter((hl.str(top_test.nearest_genes_20kb).contains('NAA11') & ((top_test.gnomad_v3.most_severe_consequence == 'missense_variant') |
                                                                            (top_test.gnomad_v3.most_severe_consequence == 'synonymous_variant'))),
                           hl.agg.count_where(top_test.GT.is_non_ref()))

pcdh10_coding = hl.agg.filter((hl.str(top_test.nearest_genes_20kb).contains('PCDH10') & ((top_test.gnomad_v3.most_severe_consequence == 'missense_variant') |
                                                                            (top_test.gnomad_v3.most_severe_consequence == 'synonymous_variant'))),
                           hl.agg.count_where(top_test.GT.is_non_ref()))

top_test = top_test.annotate_cols(non_refs_hdc = hl.agg.filter(hl.str(top_test.nearest_genes_20kb).contains('HDC'), hl.agg.count_where(top_test.GT.is_non_ref())),
                                  non_refs_chadl = hl.agg.filter(hl.str(top_test.nearest_genes_20kb).contains('CHADL'), hl.agg.count_where(top_test.GT.is_non_ref())),
                                  non_refs_maoa = hl.agg.filter(hl.str(top_test.nearest_genes_20kb).contains('MAOA'), hl.agg.count_where(top_test.GT.is_non_ref())),
                                  non_refs_naa11 = hl.agg.filter(hl.str(top_test.nearest_genes_20kb).contains('NAA11'), hl.agg.count_where(top_test.GT.is_non_ref())),
                                  non_refs_pcdh10 = hl.agg.filter(hl.str(top_test.nearest_genes_20kb).contains('PCDH10'), hl.agg.count_where(top_test.GT.is_non_ref())),
                                  non_refs_hdc_coding = hdc_coding,
                                  non_refs_chadl_coding = chadl_coding,
                                  non_refs_maoa_coding = maoa_coding,
                                  non_refs_naa11_coding = naa11_coding,
                                  non_refs_pcdh10_coding = pcdh10_coding)                     

In [ ]:
top_test.write('/net/archive/groups/plggneuromol/GTS-analysis/data/top-test.mt')

In [ ]:
#coding HDC
top_test.filter_rows((hl.str(top_test.nearest_genes_20kb).contains('HDC') & ((top_test.gnomad_v3.most_severe_consequence == 'missense_variant') |
                                                                            (top_test.gnomad_v3.most_severe_consequence == 'synonymous_variant')))).count()

In [ ]:
#coding CHADL
top_test.filter_rows((hl.str(top_test.nearest_genes_20kb).contains('CHADL') & ((top_test.gnomad_v3.most_severe_consequence == 'missense_variant') |
                                                                            (top_test.gnomad_v3.most_severe_consequence == 'synonymous_variant')))).count()

In [ ]:
#coding MAOA:
top_test.filter_rows((hl.str(top_test.nearest_genes_20kb).contains('MAOA') & ((top_test.gnomad_v3.most_severe_consequence == 'missense_variant') |
                                                                            (top_test.gnomad_v3.most_severe_consequence == 'synonymous_variant')))).count()

In [ ]:
#coding NAA11:
top_test.filter_rows((hl.str(top_test.nearest_genes_20kb).contains('NAA11') & ((top_test.gnomad_v3.most_severe_consequence == 'missense_variant') |
                                                                            (top_test.gnomad_v3.most_severe_consequence == 'synonymous_variant')))).count()

In [ ]:
#coding PCDH10:
top_test.filter_rows((hl.str(top_test.nearest_genes_20kb).contains('PCDH10') & ((top_test.gnomad_v3.most_severe_consequence == 'missense_variant') |
                                                                            (top_test.gnomad_v3.most_severe_consequence == 'synonymous_variant')))).count()

In [ ]:
#noncoding HDC
top_test.filter_rows((hl.str(top_test.nearest_genes_20kb).contains('HDC') & ((top_test.gnomad_v3.most_severe_consequence != 'missense_variant') &
                                                                            (top_test.gnomad_v3.most_severe_consequence != 'synonymous_variant')))).count()

In [ ]:
#noncoding CHADL (this should rather be 7)
top_test.filter_rows((hl.str(top_test.nearest_genes_20kb).contains('CHADL') & ((top_test.gnomad_v3.most_severe_consequence != 'missense_variant') &
                                                                            (top_test.gnomad_v3.most_severe_consequence != 'synonymous_variant')))).count()

In [ ]:
#noncoding MAOA:
top_test.filter_rows((hl.str(top_test.nearest_genes_20kb).contains('MAOA') & ((top_test.gnomad_v3.most_severe_consequence != 'missense_variant') &
                                                                            (top_test.gnomad_v3.most_severe_consequence != 'synonymous_variant')))).count()

In [ ]:
#noncoding NAA11:
top_test.filter_rows((hl.str(top_test.nearest_genes_20kb).contains('NAA11') & ((top_test.gnomad_v3.most_severe_consequence != 'missense_variant') |
                                                                            (top_test.gnomad_v3.most_severe_consequence != 'synonymous_variant')))).count()

In [ ]:
#noncoding PCDH10:
top_test.filter_rows((hl.str(top_test.nearest_genes_20kb).contains('PCDH10') & ((top_test.gnomad_v3.most_severe_consequence != 'missense_variant') |
                                                                            (top_test.gnomad_v3.most_severe_consequence != 'synonymous_variant')))).count()

In [ ]:
#all HDC
top_test.filter_rows(hl.str(top_test.nearest_genes_20kb).contains('HDC')).count()

In [ ]:
#all CHADL
top_test.filter_rows(hl.str(top_test.nearest_genes_20kb).contains('CHADL')).count()

In [ ]:
#all MAOA
top_test.filter_rows(hl.str(top_test.nearest_genes_20kb).contains('MAOA')).count()

In [ ]:
#all NAA11:
top_test.filter_rows(hl.str(top_test.nearest_genes_20kb).contains('NAA11')).count()

In [ ]:
#all PCDH10:
top_test.filter_rows(hl.str(top_test.nearest_genes_20kb).contains('PCDH10')).count()

In [ ]:
top_test.count()

In [ ]:
top_test = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/top-test.mt')

In [ ]:
to_export = top_test.cols()

In [ ]:
families = to_export.to_pandas()

In [ ]:
families

In [ ]:
families.to_csv('family_table')

In [ ]:
variants_gnomad = np.load('/net/archive/groups/plggneuromol/GTS-analysis/data/numpy/may-variants_controls5cadd10.npy')

In [ ]:
np.sum(variants_gnomad) - 1.19

In [ ]:
variants_gts_test = np.load('/net/archive/groups/plggneuromol/GTS-analysis/data/numpy/variants_gts_test5cadd10.npy') 

In [ ]:
top_test = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/top-test.mt')

In [ ]:
top_test.count()

## investigate variants that went into the model

In [ ]:
top = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/top-variants.mt')

In [ ]:
top.aggregate_rows(hl.agg.mean(top.cadd))

In [ ]:
top.aggregate_rows(hl.agg.approx_median(top.cadd))

In [ ]:
cadd_hist = top.aggregate_rows(hl.expr.aggregators.hist(top.cadd, 0, 30, 10))

p = hl.plot.histogram(cadd_hist, legend='CADD', title='CADD Histogram')
p.xaxis.axis_label_text_font_size = "15pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.axis_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
show(p)

In [ ]:
top.describe()

In [ ]:
top.aggregate_rows(hl.agg.mean(top.gnomad_v3.v3_nfe.AF))

In [ ]:
top.aggregate_rows(hl.agg.approx_median(top.gnomad_v3.v3_nfe.AF))

In [ ]:
gnomad_hist = top.aggregate_rows(hl.expr.aggregators.hist(top.gnomad_v3.v3_nfe.AF, 0, 0.2, 20))

p2 = hl.plot.histogram(gnomad_hist, legend='gnomAD MAF', title='MAF Histogram')
p2.xaxis.axis_label_text_font_size = "15pt"
p2.xaxis.major_label_text_font_size = "15pt"
p2.yaxis.axis_label_text_font_size = "15pt"
p2.yaxis.major_label_text_font_size = "15pt"

show(p2)

In [ ]:
top.count()

In [ ]:
top.aggregate_rows(hl.agg.counter(top.gnomad_v3.most_severe_consequence))

In [ ]:
top.describe()

## export top variants

In [ ]:
top = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/top-variants.mt')

In [ ]:
top.count()

In [ ]:
# check group numbers and remove extra controls

a = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-for-skat-may.mt')
b = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-test-may.mt')

samples = a.s.collect() + b.s.collect()

In [ ]:
top = top.filter_cols(hl.literal(samples).contains(top.s))

In [ ]:
#count controls and other groups

In [ ]:
top.count()

In [ ]:
top.aggregate_cols(hl.agg.filter(~top.s.contains('gnomad'), hl.agg.counter(top.phenotypes.phenotype)))

In [ ]:
top = top.annotate_rows(
                      all_gnomads_non_ref = hl.agg.filter((top.s.contains == 'gnomad'), hl.agg.count_where(top.GT.is_non_ref()))/40,
                      all_gnomads_hom_var = hl.agg.filter((top.s.contains == 'gnomad'), hl.agg.count_where(top.GT.is_hom_var()))/40,
                      controls_non_ref = hl.agg.filter((top.phenotypes.disease == 'NO'), hl.agg.count_where(top.GT.is_non_ref()))/56,
                      controls_hom_var = hl.agg.filter((top.phenotypes.disease == 'NO'), hl.agg.count_where(top.GT.is_hom_var()))/56,
                      cases_all_non_ref = hl.agg.filter((top.phenotypes.disease == 'YES'), hl.agg.count_where(top.GT.is_non_ref()))/129,
                      cases_all_hom_var = hl.agg.filter((top.phenotypes.disease == 'YES'), hl.agg.count_where(top.GT.is_hom_var()))/129)

In [ ]:
top = top.rows()
top = top.to_pandas()
top.to_csv('/net/archive/groups/plggneuromol/GTS-analysis/data/top-variants.csv')

## Validate with GWAS

1. ideentify intervals that include the genes of interest

In [ ]:
genes = hl.read_table('/net/archive/groups/plggneuromol/GTS-analysis/analysis/vcf_preprocessing/genecode_v32.ht')
genes = genes.filter(hl.is_valid_contig(genes['hg38.knownGene.chrom'], reference_genome='GRCh38'))


start = hl.cond(genes['hg38.knownGene.txStart'] < 20000, 1, genes['hg38.knownGene.txStart'] - 20000)
stop =  hl.cond(hl.contig_length(genes['hg38.knownGene.chrom'], reference_genome='GRCh38') - genes['hg38.knownGene.txEnd'] < 20000, 
                hl.contig_length(genes['hg38.knownGene.chrom'], reference_genome='GRCh38'),
                genes['hg38.knownGene.txEnd'] + 20000)

genes = genes.transmute(interval = 
                        hl.locus_interval(genes['hg38.knownGene.chrom'], 
                                          start,
                                          stop,
                                          reference_genome='GRCh38'))

In [ ]:
genes = genes.key_by(genes.interval)

In [ ]:
gwas = hl.import_table('/net/archive/groups/plggneuromol/GTS-analysis/data/TS_Oct2018', delimiter=' ')

In [ ]:
gwas = gwas.annotate(locus = hl.locus(gwas.CHR, hl.int32(gwas.BP), reference_genome='GRCh37'))

In [ ]:
rg37 = hl.get_reference('GRCh37')  
rg38 = hl.get_reference('GRCh38')  
rg37.add_liftover('/net/archive/groups/plggneuromol/GTS-analysis/analysis/references_grch37_to_grch38.over.chain.gz', rg38)

In [ ]:
gwas = gwas.annotate(hg39locus = hl.liftover(gwas.locus, 'GRCh38'))

In [ ]:
gwas = gwas.annotate(nearest_genes_20kb = hl.array(hl.set(genes.index(gwas.hg39locus, all_matches=True)['hg38.kgXref.geneSymbol'])))

In [ ]:
gwas.checkpoint('/net/archive/groups/plggneuromol/GTS-analysis/data/gwas.ht')

In [ ]:
gwas = hl.read_table('/net/archive/groups/plggneuromol/GTS-analysis/data/gwas.ht')

In [ ]:
top_genes = ['HDC', 'CHADL', 'MAOA', 'NAA11', 'PCDH10']

In [ ]:
gwas_top = gwas.filter(hl.any(lambda x: hl.literal(top_genes).contains(x), gwas.nearest_genes_20kb))

In [ ]:
p_top = gwas_top.P.collect()

In [ ]:
p_hist = gwas_top.aggregate(hl.expr.aggregators.hist(hl.float64(gwas_top.P), 0, 1, 50))

p = hl.plot.histogram(p_hist, legend='p', title='p_value_histogram')
p.xaxis.axis_label_text_font_size = "15pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.axis_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
show(p)

In [ ]:
genes_background = np.load('/net/archive/groups/plggneuromol/GTS-analysis/analysis/numpy/genes_background.npy')

In [ ]:
p_values = []
    
for p in range(100):
    randoms = np.random.randint(len(genes_background), size=5)
    genes = [j for i, j in enumerate(genes_background) if i in randoms]
        
    p_value = gwas.filter(hl.any(lambda x: hl.literal(genes).contains(x), gwas.nearest_genes_20kb)).P.collect()
    
    p_values.append(p_value)

In [ ]:
y = [float(i) for i in p_top]
y = -np.log10(y)
hist_top, edge_top = np.histogram(y, bins = 50)

In [ ]:
import bokeh.palettes
from bokeh.models import CategoricalColorMapper

colors = bokeh.palettes.plasma(200)

In [ ]:
histograms = []
edges = []

for p in p_values:
    x = [float(i) for i in p]
    x = -np.log10(x)
    hist,edge = np.histogram(x, bins = 50)
    histograms.append(hist)
    edges.append(edge)

In [ ]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

p1 = figure(plot_width=800, plot_height=800)

for x in range(100):
    p1.quad(top=np.log10(histograms[x]), bottom=0, left=edges[x][:-1], right=edges[x][1:],
         fill_color=colors[100], line_color=colors[50], alpha=0.05)
    
p1.quad(top=np.log10(hist_top), bottom=0, left=edge_top[:-1], right=edge_top[1:],
         fill_color=colors[0], line_color=colors[0], alpha=1)

p1.xaxis.axis_label = '-log10(p_value)'
p1.yaxis.axis_label = 'log10(number of SNPs)'

p1.xaxis.axis_label_text_font_size = "15pt"
p1.xaxis.major_label_text_font_size = "15pt"
p1.yaxis.axis_label_text_font_size = "15pt"
p1.yaxis.major_label_text_font_size = "15pt"

show(p1)

In [ ]:
p_all = []

for p in p_values:
    x = [float(i) for i in p]
    x = -np.log10(x)
    p_all.append(x)

In [ ]:
import itertools

p_all = (list(itertools.chain.from_iterable(p_all)))

In [ ]:
np.percentile(p_all, 99)

In [ ]:
-np.log10(0.008)

In [ ]:
y.shape

In [ ]:
np.sum(y > 2.09)#/np.shape(y)

### For large families see if variants in additional genes segregate.
Rule : at least 60% of individuals with GTS are non-ref and at less than 30% of controls are ref. CADD > 10


In [ ]:
mt_test = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/mt_test_10.mt')

In [ ]:
families = ['A','B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'W', 'X', 'Y', 'Z']

In [ ]:
mt_test.count()

variants with CADD > 10 that occur in any of the larger families

In [ ]:
fams = mt_test.aggregate_cols(hl.agg.counter(mt_test.phenotypes.family))

16 families with over 5 individuals are included:

In [ ]:
list(np.array(list(fams.keys()))[np.array(list(fams.values())) > 4])

each of the families should have at least two individuals with tics or GTS:

In [ ]:
gts = mt_test.aggregate_cols(hl.agg.group_by(mt_test.phenotypes.family, hl.agg.counter(mt_test.phenotypes.disease)))

In [ ]:
gts = { your_key: gts[your_key] for your_key in list(np.array(list(fams.keys()))[np.array(list(fams.values())) > 4]) }

In [ ]:
gts

In [ ]:
mt_test.aggregate_cols(hl.agg.counter(mt_test.phenotypes.disease))

In [ ]:
mt_test.aggregate_cols(hl.agg.counter(mt_test.phenotypes.phenotype))

### additionally we will remove families with only 1 or no controls :  U, H

In [ ]:
fams_to_test = ['E', 'T', 'Y', 'J', 'F', 'A', 'I', 'B', 'C', 'W', 'R', 'D', 'S', 'X']

In [ ]:
genes = ['DCC', 'RBFOX', 'SLC30A9', 'DCAF4L1', 'SORCS3', 'KCNQ5', 'KCNQ-IT1', 'APOPT1', 'C14orf2', 'NAA11', 'NEGR1',
        'CHADL', 'SOX5', 'PANK2', 'COL27A1', 'PDGFB', 'CELSR3', 'OPA1', 'FBN2', 'WWC1', 'NIPBL', 
             'FN1', 'FBN2', 'SLITRK1', 'SLITRK2', 'SLITRK3', 'SLITRK4', 'SLITRK5', 'SLITRK6', 
             'HDC', 'OPRK1', 'PCDH10', 'NTSR2', 'OPRK1', 'CHD8', 'SCUBE1', 'PNKD', 'CNTNAP2', 'MOG', 
             'DRD2', 'DRD3', 'DRD4', 'DRD5', 'DAT1', 'DBH', 'HTR2A', 'TPH2', 'EAAT1', 'SAPAP3',
            'CTNNA3', 'NLGN4', 'FSCB', 'IMMP2L', 'NRXN1', 'AADAC', 'DBH', 'MAOA', 'HTR1A', 'HTR2C', 'SLC6A4',
             'TPH2', 'COL27A1', '5-HTTLPR', 'EAAT1', 'COL8A1', 'KCNE1', 'KCNE2',
         'RICTOR', 'WWC1', 'CELSR3, NIPBL', 'FN1', 'PNKD', 'CDH26', 'CADM2', 'OPCML', 'CDH9',
         'NCAM2', 'CD47', 'CDH5', 'CADM4', 'C1QBP', 'CTTN', 'LSAMP',
         'PKP4', 'PCDH1', 'CNTNAP2', 'MBP', 'GABBR2', 'GABBR2', 'GRIK4', 'NCR1', 'FLT3', 'IL12A', 'HDAC9',
         'CD180', 'CDH26', 'NCAM2', 'NTM', 'ROBO2']

In [ ]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/mt_test_10.mt')

In [ ]:
mt.count()

In [ ]:
mt = mt.filter_rows(hl.any(lambda x: hl.literal(genes).contains(x), mt.nearest_genes_20kb))
mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))

In [ ]:
mt.checkpoint('/net/archive/groups/plggneuromol/GTS-analysis/data/mt_86_genes_cadd_10.mt')

In [ ]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/mt_86_genes_cadd_10.mt')

In [ ]:
mt.count()

In [ ]:
mt = mt.filter_cols(hl.set(fams_to_test).contains(mt.phenotypes.family))
mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))

In [ ]:
mt.checkpoint('/net/archive/groups/plggneuromol/GTS-analysis/data/mt_86_genes_cadd_10_fams.mt')

In [ ]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/mt_86_genes_cadd_10_fams.mt')

In [ ]:
mt_all = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/GTS-with-pheno.mt')

In [ ]:
#reasigned phenotypes:
pheno = hl.import_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/pheno-and-gene-lists/GTS-coded-corrected-june-2021.csv', delimiter=',', impute=True,  quote='\"')

In [ ]:
pheno = pheno.key_by(pheno.ID)

In [ ]:
mt_all = mt_all.annotate_cols(phenotypes = pheno[mt_all.s])

In [ ]:
mt_all.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/GTS-with-pheno-step1.mt')

In [ ]:
mt_all = mt_all.annotate_rows(cases_with_gts = hl.agg.group_by(mt_all.phenotypes.family, hl.agg.count_where(mt_all.phenotypes.disease == 'YES')),
                      variants_with_gts = hl.agg.group_by((mt_all.phenotypes.family, mt_all.phenotypes.disease), hl.agg.count_where(mt_all.GT.is_non_ref())),
                      carriers_in_families = hl.agg.group_by(mt_all.s, hl.agg.count_where(mt_all.GT.is_non_ref())),
                      variants_in_sporadic_cases = hl.agg.group_by((mt_all.phenotypes.family, mt_all.phenotypes.disease), hl.agg.counter(mt_all.GT.is_non_ref()))
                     )

mt_all = mt_all.annotate_rows(sporadic_cases_non_refs = hl.if_else(mt_all.variants_in_sporadic_cases[('.','YES')].contains(True), 
                                                                mt_all.variants_in_sporadic_cases[('.','YES')][True],
                                                                0),
                             sporadic_cases_refs = hl.if_else(mt_all.variants_in_sporadic_cases[('.','YES')].contains(False), 
                                                                mt_all.variants_in_sporadic_cases[('.','YES')][False],
                                                                0))

mt_all = mt_all.annotate_rows(variants_in_familial = hl.agg.group_by((mt_all.phenotypes.disease), hl.agg.counter(mt_all.GT.is_non_ref())))

mt_all = mt_all.annotate_rows(controls_non_refs = hl.if_else(mt_all.variants_in_familial['NO'].contains(True), 
                                                                mt_all.variants_in_familial['NO'][True],
                                                                0),
                              controls_refs = hl.if_else(mt_all.variants_in_familial['NO'].contains(False), 
                                                                mt_all.variants_in_familial['NO'][False],
                                                                0),
                              cases_non_refs = hl.if_else(mt_all.variants_in_familial['YES'].contains(True), 
                                                                mt_all.variants_in_familial['YES'][True],
                                                                0),
                              cases_refs = hl.if_else(mt_all.variants_in_familial['YES'].contains(False), 
                                                                mt_all.variants_in_familial['YES'][False],
                                                                0))

In [ ]:
mt_all.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/GTS-with-pheno-all.mt')

In [ ]:
mt_all = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/GTS-with-pheno-all.mt')

In [ ]:
mt_all.count()

In [ ]:
fams = []
fam_hts = []
for f in fams_to_test:
    fam = mt.filter_cols(mt.phenotypes.family == f)
    fam = fam.filter_rows((hl.agg.any(fam.GT.is_non_ref())))
    fam = fam.annotate_rows(variants = hl.agg.group_by((fam.phenotypes.disease), hl.agg.count_where(fam.GT.is_non_ref())))
    fam = fam.filter_rows(((fam.variants['NO']/gts[f]['NO']) < 0.2) & ((fam.variants['YES']/gts[f]['YES']) > 0.7))
    fam = fam.annotate_rows(cases_refs = mt_all.index_rows(fam.row_key).cases_refs,
                            cases_non_refs = mt_all.index_rows(fam.row_key).cases_non_refs,
                            controls_refs = mt_all.index_rows(fam.row_key).controls_refs,
                            controls_non_refs = mt_all.index_rows(fam.row_key).controls_non_refs)
    fam = fam.filter_rows((fam.cases_non_refs/(fam.cases_refs + fam.cases_non_refs) > (fam.controls_non_refs/(fam.controls_refs + fam.controls_non_refs))))
    fam = fam.filter_rows(fam.gnomad_v3.v3_nfe.AF < 0.05)
    fam = fam.annotate_rows(non_refs_in_fam = hl.agg.group_by((fam.phenotypes.phenotype), hl.agg.count_where(fam.GT.is_non_ref())),
                            refs_in_fam = hl.agg.group_by((fam.phenotypes.phenotype), hl.agg.count_where(fam.GT.is_hom_ref())),
                            samples = hl.agg.filter(fam.GT.is_non_ref(), hl.agg.collect(fam.s))
                           )
    fam = fam.annotate_rows(non_refs_fam_controls = fam.non_refs_in_fam['.'],
                            non_refs_fam_gts = fam.non_refs_in_fam['GTS'],
                            non_refs_fam_tics = hl.cond(fam.non_refs_in_fam.contains('tics'), 
                                                                fam.non_refs_in_fam['tics'],
                                                                0),
                            refs_fam_controls = fam.refs_in_fam['.'],
                            refs_fam_gts = fam.refs_in_fam['GTS'],
                            refs_fam_tics = hl.cond(fam.refs_in_fam.contains('tics'), 
                                                                fam.refs_in_fam['tics'],
                                                                0)
                           )
    fam = fam.rows()
    fam = fam.select(fam.rsid, fam.gnomad_v3.v3_nfe.AF, fam.gnomad_v3.v3_nfe.homozygote_count, fam.gnomad_v3.most_severe_consequence,
                    fam.hpo, fam.cadd, fam.nearest_genes_20kb, fam.cases_refs, fam.cases_non_refs, fam.controls_refs, fam.controls_non_refs,
                    fam.samples, fam.non_refs_fam_controls, fam.non_refs_fam_gts, fam.non_refs_fam_tics, fam.refs_fam_controls, fam.refs_fam_gts, fam.refs_fam_tics)
    fam_hts.append(fam)
    
    
    fam = fam.to_pandas()
    fams.append(fam)
    fam.to_csv('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/results/fam-analysis-'+f+'variants.csv')

In [ ]:
len(fam_hts)

In [ ]:
len(fams_to_test)

In [ ]:
fam_hts[0].describe()

In [ ]:
gene_dict = {}
for idx, fam in enumerate(fams_to_test):
    gene_dict[fam] = fam_hts[idx].nearest_genes_20kb.collect()

In [ ]:
for key, value in gene_dict.items():
    gene_dict[key] = (set(itertools.chain(*value)))

In [ ]:
goi = {}
for key, value in gene_dict.items():
    new_value = []
    for i in value:
        if i in genes:
            new_value.append(i)
    goi[key] = new_value

In [ ]:
top_genes = list(itertools.chain(*goi.values()))

In [ ]:
cons_dict = {}
for idx, fam in enumerate(fams_to_test):
    cons_dict[fam] = fam_hts[idx].most_severe_consequence.collect()

In [ ]:
from collections import Counter

In [ ]:
Counter(list(itertools.chain(*cons_dict.values())))

In [ ]:
for f in fam_hts:
    print(f.count())

In [ ]:
from collections import Counter
df = pd.DataFrame(list(Counter(top_genes).items()),columns = ['column1','column2'])

In [ ]:
df[(df['column2'] > 2)]

In [ ]:
df = pd.DataFrame(list(goi.items()),columns = ['family','genes'])

In [ ]:
df.to_csv('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/results/familial_genes_summary.csv')